<small><i>This notebook was put together by [Abel Meneses](http://www.menesesabad.com) for PyData 2018. Source and license info is on [GitHub](https://github.com/sorice/nlp_pydata2018/).</i></small>

# Bases de NLP

[Ver proceso previo: Normalización de Textos basado en Expresiones Regulares](02.3-Text-Normalization.ipynb)

**Fecha de elaboración inicial**: 8 de agosto de 2015
**Última actualización**: 
<a id='indice'></a>
## Índice

1. [Datos.](#datos) Descripción de los datos que se utilizarán en el notebook.

- 1.1 [Transformar los datos.](#transformar_los_datos) de PDF a txt para hacer más fácil su
manipulación en python.

2. [Ejemplos Básicos.](#ejemplos_basicos) Usualmente útiles para hacer Minería de Texto.

3. [Operaciones usando elementos estadísticos.](#operaciones_estadisticas)

- 3.1 [Filtrado de Stopwords](#filtrado_de_stopwords)
- 3.2 [Stemming](#stemming)
- 3.3 [Lemmatizacion](#lematizacion)
- 3.4 [N-gramas](#n-gramas)

[Conclusiones](#conclusiones)

[Ejercicios](#ejercicios)

[Referencias](#referencias)

[Índice Alfabético](#indice_alfabetico)

<a id='datos'></a>
## Datos

Para todos los ejemplos se usarán dos libros, uno en inglés y otro en español. El 
primero es **Free Software Free Society** y el segundo una traducción de este primero
**Software Libre para una Sociedad Libre**.

<a id='transformar_los_datos'></a>
### Transformar los Datos
Generalmente casi todos los materiales que poseemos son *PDFs* y para operar con textos
en python lo mejor es usar .txt o formatos no enriquecidos. ¿Cómo transformar PDF
en TXT?

Nuestra recomendación es usar pdftotxt que aparece en los repositorios de GNU/Linux
dentro del paquete **poppler-utils**. Si conocen algún escript adecuado para esta tarea
utilizando la biblioteca *ghostscript* la recomendamos por encima de *pdftotext*. Sin 
embargo es bastante difícil encontrar semejante script, no dudamos de que en el futuro
lo encontremos. El comando a ejecutar es sencillo:

~$ pdftotext archivo.pdf

**Resultado:** archivo.txt

Hay que tener en cuenta que este script de *extracción de textos* genera los .txt
con muchos problemas: fundamentalmente caracteres extraños.
Tal vez resultará más útil para el lector estudiar aplicaciones y bibliotecas más 
especializadas para este tipo de problema del procesamiento de textos como: Apache-Tika,
u otros. Sin embargo en el tema anterior sobre pre-procesamiento, el lector podrá
encontrar como resolver estos problemas sin usar grandes bibliotecas
para un buen trabajo con los ejemplos básicos de NLP que se ofrecen a continuación.

<a id='ejemplos_basicos'></a>
## Ejemplos básicos al estilo del libro de NLTK

Una de las primeras cosas interesantes de un texto, es saber cuantas palabras contiene 
un texto.

In [1]:
#SoftwareWars.txt is generated with pdftotext from SoftwareWars.pdf
texto = open('data/SoftwareWars.txt').read()
words = 0
for line in texto.split('\n'):
    for word in line.split():
        words +=1
print ("Total de palabras: ", words)

Total de palabras:  95976


In [80]:
#SoftwareWars2.txt is a normalized version of SoftwareWars.txt
texto = open('data/SoftwareWars2.txt').read().lower()
words = 0
for line in texto.split('\n'):
    for word in line.split():
        words +=1
print ("Total de palabras: ", words)

Total de palabras:  102721


Un pequeño ejemplo para contar oraciones. _data/Avello2016n_ es una tesis normalizada con el notebook anterior. Este texto de la UCLV ha sido liberado bajo Creative Commons ([Avello2016](#Avello2016)) .

In [79]:
tesis = open('data/Avello2016n.txt').read()
for line in tesis.split('\n'):
    for i,sentence in enumerate(line.split('.')):
        count = i
print (count)
        

2032


Ejemplo con **NLTK**.

In [81]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("\s+", gaps=True)
tokens = tokenizer.tokenize(texto)
print ("Total de palabras: ", len(tokens))

Total de palabras:  102721


En ambos casos se devuelve el total de palabras(o tokens) divididos por el caracter
*espacio*. Sin embargo las palabras en un texto se repiten. ¿Cómo saber las palabras
únicas?

In [5]:
tokens_unique=set([])
tokens_unique = set(tokens)
print ("Palabras únicas:", len(tokens_unique))

Palabras únicas: 3082


Como se puede ver en el libro Bird et al.[[1](#Bird2009)] la pregunta más simple y 
común que se hace uno al ver estas cifras suelen ser: ¿Cuál es el promedio de palabras
por página? ¿Cuál es la palabra más utilizada? ¿Qué palabras se usan una vez? Veamos
algunas formas de calcularlo, para ello necesitaremos algunas funciones extras:

<a id='sect2.5'></a>

In [6]:
#Inicializar un diccionario para guardar el # de apariciones de cada palabra.
dict = {}
for word in tokens_unique:
    dict[word]=0
#Diccionario con word = # apariciones.
for token in tokens:
    dict[token]+=1
#Operar con una tupla puede ser mejor. Lista([#apariciones,word])
tupla = []
for word in dict:
    tupla.append([dict[word],word])

In [7]:
#Ver 5 palabras aleatoriamente contenidas en el diccionario.
import random
for i in random.sample(range(len(dict)),5):
    print (tupla[i][1],":",tupla[i][0])

HU_07 : 1
cinco : 1
casos : 1
conforman : 1
Carly : 1


Esta tupla puede ser ordenada de la siguiente forma: al poner el # de apariciones 
delante, podemos usar el elemento *tupla[i][0]* como parámetro de ordenamiento.

In [8]:
tupla=sorted(tupla)
print ("Las 5 palabras más utilizadas son:")
for i in range(1,6):
    print (tupla[-i][1],":",tupla[-i][0])

Las 5 palabras más utilizadas son:
. : 2032
de : 1091
la : 578
en : 348
el : 275


In [9]:
print ("5 palabras que se usan una vez:")
for i in range(5):
    print (tupla[i][1],":",tupla[i][0])
#Número de páginas del libro After the Software War = 300
print ("Promedio de palabras por página", len(tokens)/300)

5 palabras que se usan una vez:
#m : 1
#p : 1
00002942 : 1
00003133 : 1
00078760 : 1
Promedio de alabras por página 56.733333333333334


In [10]:
print (dict["software"])

19


Volver al [*Índice*](#indice).

<a id='operaciones_estadisticas'></a>
# Operaciones usando elementos estadísticos

<a id='filtrado_de_stopwords'></a>
## Filtrado de Stopwords

Como habremos podido observar en la respuesta a *"las palabras más usadas"*, la mayoría
de ellas son palabras que no contienen significado o no son ni verbos, ni sustantivos, 
ni adjetivos. En NLP se acostumbra a eliminar estas palabras para procesar el resto más
significativo. Para hacerlo necesitamos normalmente un fichero de texto con los 
stopwords que por lo general y por convención se han definido o calculado. 
En nuestro caso usaremos los de NLTK.Una versión personalizada por el autor también
se usa, hecha a partir de los originales de *NLTK* y 
[Snowball](http://snowball.tartarus.org/texts/introduction.html)

**Nota**: otro elemento contenido en el siguiente algoritmo es la eliminación de las 
palabras con longitud = 1, todas palabras sin significado.

In [11]:
import time
from nltk.corpus import stopwords
timei = time.time()
english_stops = set(stopwords.words('en'))
tokens_afterstops=[]
for k in range(len(tokens)-1):
    if tokens[k] not in english_stops and len(tokens[k])>1:
        tokens_afterstops.append(tokens[k])
timef = time.time()-timei
print ("Tiempo de filtrado de stopwords: ",timef)

tokens_unique1 = set(tokens_afterstops)
dict1 = {} #dict con keys = set de tokens after stops
for word in tokens_unique1:
    dict1[word]=0

tupla1 = [] #Creando lista de tuplas(# ocurrencias,word) sin stopwords
for token in tokens_afterstops:
    dict1[token]+=1
for word in dict1:
    tupla1.append([dict1[word],word])
tupla1=sorted(tupla1)

Tiempo de filtrado de stopwords:  0.017447471618652344


In [12]:
print ("Palabras únicas sin stops:", len(tokens_unique1))
print ("Las 5 palabras más utilizadas después de filtrar los stopwords son:")
for i in range(1,6):
    print (tupla1[-i][1],":",tupla1[-i][0])
print ("Total de palabras del texto: ", len(tokens))
print ("Palabras en el texto sin stopwords:", len(tokens_afterstops))
print ("Palabras eliminadas en el proceso de filtrado de stopwords:", 
len(tokens)-len(tokens_afterstops))

Palabras únicas sin stops: 3002
Las 5 palabras más utilizadas después de filtrar los stopwords son:
de : 1091
la : 578
en : 348
el : 275
se : 213
Total de palabras del texto:  17020
Palabras en el texto sin stopwords: 13912
Palabras eliminadas en el proceso de filtrado de stopwords: 3108


Volver al [*Índice*](#indice).

<a id='stemming'></a>
## Stemming 

Proceso mediante el cual se eliminan de la palabra los “morfemas”, utilizando reglas 
predefinidas que se corresponden con las terminaciones más comunes de las palabras en 
un idioma. Trabaja la morfología de las palabras.

In [17]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
words_stops = len(tokens) #Longitud de tokens después de los dos primeros pasos.
tokens_stem=[]
for i in range(len(tokens_afterstops)):
    tokens_stem.append(stemmer.stem(tokens_afterstops[i]))
        #print i

timei = time.time()
stemmer = PorterStemmer()
words_stops = len(tokens) #Longitud de tokens después de los dos primeros pasos.
tokens_stem=[]
for i in range(len(tokens)):
    tokens_stem.append(stemmer.stem(tokens[i]))
        #print i
timef = time.time()-timei
print ("Tiempo de steeming: ",timef)

print (tokens_afterstops[27],":",tokens_stem[27])
tokens_unique2 = set(tokens_stem)

Tiempo de steeming:  0.09572577476501465
Ing : Alexand


In [18]:
print ("Palabras reducidas durante el steaming: de %d se redujeron en %d." 
       % (len(tokens_unique1), len(tokens_unique1)-len(tokens_unique2)))
print ("Palabras únicas tras el steamming: %d." % (len(tokens_unique2)))

Palabras reducidas durante el steaming: de 3002 se redujeron en 184.
Palabras únicas tras el steamming: 2818.


Volver al [*Índice*](#indice).

<a id='lematizacion'></a>
## Lematización

Proceso mediante el cual se extrae el “lexema” de la palabra. Generalmente es necesario
utilizar una base de datos (BD) que contenga información de los lexemas o lemas (como 
también se le suele llamar a los lexemas), estas BD son generalmente semánticas. 
Trabaja la morfología de las palabras.

En el siguiente ejemplo se utiliza Wordnet y su implementación en NLTK. **Wordnet** es 
una de las BD semánticas más importantes creadas por la humanidad cuyas versiones en 
inglés son licenciadas bajo principios libres. Se puede encontrar varias versiones de ella
en los diferentes repositorios de linux. Una versión profesional de Wordnet en español
existe pero es comercial y su costo es de más de 5000 euros.

In [19]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#Lematizando.
dict_lem = {}
timei=time.time()
for i in range(len(tokens)):
    dict_lem[tokens[i]] = lemmatizer.lemmatize(tokens[i])
timef=time.time()-timei
print("Tiempo de lematización:", timef)

#Construyendo un diccionario con los términos únicos dict_lem after stops
#y una lista con estos mismos términos.

dict_lem = {}
for token in tokens_unique1:
    dict_lem[token] = token
tokens_uniqueB = list(dict_lem.keys()) #ojo: en py3 el método keys() devuelve un objeto de tipo dict_keys (no-list) y no indexable.

#Lematizando los términos únicos.
for i in range(len(tokens_uniqueB)):
    dict_lem[tokens_uniqueB[i]] = lemmatizer.lemmatize(tokens_uniqueB[i])

tokens_lem = tokens_afterstops.copy()
for i in range(len(tokens_afterstops)):
    tokens_lem[i] = dict_lem[tokens_afterstops[i]]
        
tokens_unique3 = set(tokens_lem)

Tiempo de lematización: 2.6319186687469482


In [20]:
setDiff = tokens_unique1.union(tokens_unique3) - tokens_unique1.intersection(tokens_unique3)

count = 10
for i in range(len(tokens_lem)):
    if tokens_lem[i] != tokens_afterstops[i]:
        print (tokens_afterstops[i],":",tokens_lem[i])
        count-=1
        if count < 1:
            break
print ("Palabras reducidas durante la Lematización de %d se redujeron en %d" 
       % (len(tokens_unique1), len(tokens_unique1)-len(tokens_unique3)))
print ("Palabras únicas tras el steamming:", len(tokens_unique3))

mes : me
las : la
fines : fine
genera : genus
las : la
morales : morale
es : e
todas : toda
las : la
personas : persona
Palabras reducidas durante la Lematización de 3002 se redujeron en 17
Palabras únicas tras el steamming: 2985


Es importante destacar que tras el stemming quedaron 2818 palabras y tras la 
lematización quedaron 2985, en ambos casos se parte del resultado del filtrado
de stopwords. En ambos casos también estamos en presencia de una **reducción de la
dimensionalidad**, o sea disminuir el tamaño de los datos a analizar perdiendo la menor
cantidad de información.
Sin embargo el autor recomienda utilizar las 2985 palabras de la lematización porque es
un proceso donde se puede regresar a la palabra original, el stemming hace un 
tronchado de las palabras que es irreversible. Sí hay que notar que la lematización es
más lenta, pero tiene en cuenta para generar cada lema la función POS de cada palabra
*(o sea sí esta es un verbo, sustantivo, adjetivo o adverbio, una misma palabra puede
tener lemas diferentes en función de su POS)*.

Volver al [*Índice*](#indice).

<a id='n-gramas'></a>
## N-gramas

La técnica de n-gramas es muy simple, consiste en dividir el texto tanto en caracteres
como en palabras y hacer nuevos tokens compuestos por n-caracteres o n-palabras. En 
este ejemplo veremos una división en *n-palabras*. Basadas en las propiedades de **Los
Modelos ocultos de Markov** esta nueva lista, para $n \geqq 3$ suele ser mucho más útil 
que las palabras simples. La func **ngrams2** fue el segundo diseño elaborado en abril
de 2015 para el algoritmo de detección de texto reusado (del autor) en su primera 
versión. La versión inicial del 2013 se incluye en la sección de performance.

In [1]:
def ngrams2(text,n):
    tokens=[];ngram="";ngram_list=[]
    for word in text.split():
        tokens.append(word)
    if n >= len(tokens):
        raise Exception("Not possible, n most be shorter than total words.")
    for i in range(len(tokens)-n+1):
        for j in range(i,i+n):
            ngram+=tokens[j]+" "
        ngram_list.append(ngram)
        ngram=""
    return ngram_list

texto = open('data/SoftwareWars2.txt').read()
ngramsl=ngrams2(texto,3)
print ("Total de n-gramas: %d." % len(ngramsl))
ngramsl_unique = set(ngramsl)
print ("N-gramas únicos: %d." % len(ngramsl_unique))

Total de n-gramas: 102719.
N-gramas únicos: 89823.


A continuación incluyo una actualización de este algoritmo con conocimientos aprendidos
en marzo del 2016 al estudiar high performance in python [[3](#Gorelick2014)]. Implementando códigos 
pythonicos utilizando estructuras optimizadas (deque) y la función join de la clase str.

In [2]:
from collections import deque

def ngrams4(text,n):
    tokens=[];ngram_list=deque()
    for word in text.split():
        tokens.append(word)
    if n >= len(tokens):
        raise Exception("Not possible, n most be shorter than total words.")
    for i in range(len(tokens)-n+1):
        ngram_tokens = [tokens[j] for j in range(i,i+n)]
        ngram =" ".join(ngram_tokens)
        ngram_list.append(ngram)
    return ngram_list

texto = open('data/SoftwareWars2.txt').read()
ngramsl=ngrams4(texto,3)
print ("Total de n-gramas: %d." % len(ngramsl))
ngramsl_unique = set(ngramsl)
print ("N-gramas únicos: %d." % len(ngramsl_unique))

Total de n-gramas: 102719.
N-gramas únicos: 89823.


In [15]:
print (ngramsl[26])

digital download .


Función ngrams basada en iteradores, el retorno **yield** y las función interna de las listas __iadd__. Elaborado el 
3 de septiembre de 2016.
Como se verá en la sección de [*Profiling*](#profiling) esta implementación es la más rápida de todas(10x) respecto a la
original del 2013.
La idea de esta función salió de revisar la implementación del método *_split* de la clase **NGram** del módulo 
[ngram](http://github.com/gpoulter/python-ngram) de Graham Poulter, el mismo es incluído dentro de los recursos 
de este tema para una revisión más detallada por parte de los estudiantes.

In [5]:
def ngram_split(text,n):
    ngram = ''
    gram_count = 0
    for i,word in enumerate(text.split(),1):
        if gram_count-n == -1 and i > n:
            ngram = ngram[ngram.find(' ')+1:]
        ngram += word+' '; gram_count+=1
        if gram_count == n:
            gram_count -= 1
            yield ngram
        
def ngrams5(text,n):
    ngrams = []
    ngrams.__iadd__(ngram_split(text,n))
    if len(ngrams) > 0:
        return ngrams
    else:
        raise Exception("Not possible, n is longer than total words.")

ngrams = ngrams5(texto,3)
print ("Total de n-gramas: %d." % len(ngrams))
ngramsl_unique = set(ngramsl)
print ("N-gramas únicos: %d." % len(ngramsl_unique))

Total de n-gramas: 102719.
N-gramas únicos: 89823.


En la sección [*Profiling*](#profiling) se analiza el performance de estas 
implementaciones, y el de otras bibliotecas fundamentales de python para NLP. Al 
concluir se incluye una sección [NLTK ngrams](#nltk_ngrams) para evaluar la dificultad
de usar esta función en NLTK.

A continuación otra función implementada en el software [Takelab](http://takelab.fer.hr/)
un sistema creado en python para el evento SEMEVAL de evaluación semántica de textos. Utiliza también iteradores
pero usando la función *zip* y la estructura optimizada *deque*, vista ya en un ejemplo anterior.
Esta implementación se estudió y elaboró el 18 de octubre del 2017 durante el desarrollo de los paquetes
[preprocess](https://github.com/sorice/preprocess) y del paquete [textsim](https://github.com/sorice/textsim)
ambos necesarios para los experimentos de Reconocimiento de Paráfrasis y Detección de Texto Reusado elaborados
por el autor.

In [8]:
from collections import deque

def make_ngrams(l, n):
    """Ngrams generation func."""
    rez = [l[i:(-n + i + 1)] for i in range(n - 1)]
    rez.append(l[n - 1:])
    return zip(*rez)

def ngrams6(text,n):
    A = make_ngrams(text.split(),3)
    return deque(A)
    
ngramsl = ngrams6(texto,3)
print ("Total de n-gramas: %d." % len(ngramsl))
ngramsl_unique = set(ngramsl)
print ("N-gramas únicos: %d." % len(ngramsl_unique))

Total de n-gramas: 102719.
N-gramas únicos: 89823.


Volver al [*Índice*](#indice).

<a id='profiling'></a>
## Profiling

Vamos a comparar este método de Abel de n-grams del 2015, el ngram_index del 2013
y el método *ngrams* de la biblioteca belga **Patterns**.

In [11]:
from time import clock

from scripts.tokens import ngram_index as ngrams1
start_time2 = clock()
ngramsa = ngrams1(texto,3)
end_time2 = clock()-start_time2

start_time1 = clock()
ngramsb = ngrams2(texto,3)
end_time1 = clock()-start_time1

"""Esta sección solo se puede usar en py2, pues la 
biblioteca Pattern no tiene soporte para py3."""
#from pattern.en import ngrams as ngrams3
#start_time3 = clock()
#ngramsc = ngrams3(texto,n=3)
#end_time3 = clock()-start_time3

start_time4 = clock()
ngramsd = ngrams4(texto,3)
end_time4 = clock()-start_time4

start_time5 = clock()
ngramse = ngrams5(texto,3)
end_time5 = clock()-start_time5

start_time6 = clock()
ngramsf = ngrams6(texto,3)
end_time6 = clock()-start_time6

print ('Función de ngrams nov/2013: %.4f' % end_time2)
%timeit ngrams1(texto,3)
print (len(ngramsb))

print ('Función de ngrams abril/2015: %.4f' % end_time1)
%timeit ngrams2(texto,3)
print (len(ngramsa))

#Patter is only active for python2, still working on this
#print ('Función de ngrams Pattern octubre/2013: %.4f' % end_time3)
#%timeit ngram3(texto,3)
#print (len(ngramsc))

print ('Función de ngrams marzo/2016: %.4f' % end_time4)
%timeit ngrams4(texto,3)
print (len(ngramsd))

print ('Función de ngrams septiembre/2016: %.4f' % end_time5)
%timeit ngrams5(texto,3)
print (len(ngramse))

print ('Función de ngrams octubre/2017: %.4f' % end_time5)
%timeit ngrams6(texto,3)
print (len(ngramsf))

Función de ngrams nov/2013: 0.8638
1 loops, best of 3: 781 ms per loop
102719
Función de ngrams abril/2015: 0.1403
10 loops, best of 3: 123 ms per loop
102719
Función de ngrams marzo/2016: 0.1565
10 loops, best of 3: 145 ms per loop
102719
Función de ngrams septiembre/2016: 0.1173
10 loops, best of 3: 102 ms per loop
102719
Función de ngrams octubre/2017: 0.1173
10 loops, best of 3: 27.7 ms per loop
102719


In [12]:
print (ngramsa[26])
print (ngramsb[26])
print (ngramsd[26])
print (ngramse[26])
print (ngramsf[26])

 digital download .
digital download . 
digital download .
digital download . 
('digital', 'download', '.')


<a id='nltk_ngrams'></a>
## NLTK Ngrams

**Nota:** NLTK tiene una función de ngramas
     
    from nltk import ngrams

Sin embargo en la versión 3 esta función es un generador, y como entrada hay que pasar
una secuencia de palabras (una lista) y no el texto original. Ejemplo para obtener un resultado similar al que
hemos obtenido con las 4 funciones anteriores hacemos lo siguiente:

In [30]:
from nltk import ngrams as ngramsNLTK
from nltk.tokenize import word_tokenize
texto = open('SoftwareWars2.txt').read().lower()

def ngramsNltk(texto,number):
    ngramse =[]
    token_text = word_tokenize(texto)
    ingramse = ngramsNLTK(token_text,number)
    for i in ingramse:
        ngramse.append(i)
    return ngramse

start_time_nltk = clock()
ngramsnltk = ngramsNltk(texto,3)
end_time_nltk = clock()-start_time_nltk
print ('Función de ngrams NLTK octubre/2015: %.4f' % end_time_nltk)
%timeit ngramsNltk(texto,3)
print (len(ngramsnltk))

Función de ngrams NLTK octubre/2015: 1.0388
1 loops, best of 3: 1 s per loop
102719


Como puede observarse aquí el resultado es más lento que en cualquiera de los anteriores
fundamentalmente por el uso de dos funciones (tokenizar y luego ngramas). En general
es más lenta que la mejor solución local.

<a id='conclusiones'></a>

## Conclusiones
- El procesamiento de textos es un mundo fascinante y de enormes utilidades.
- En python la biblioteca **NLTK** es un software maduro, aunque con muchas
insuficiencias para el idioma español.
- La función de ngramas local basada en iteradores, zip y deque es 17x la original del 2013.
- **NLTK** puede ser optimizada, sí comparamos casos como la función de ngrams.
- Las técnicas fundamentales para descomponer un texto se basan en estadística.
- Wordnet y otros recursos como Babelfish suelen dar excelentes resultados para 
procesamiento de la lengua basado en conocimiento. Sin embargo estos métodos pueden
consumir grandes cuotas de HD, CPU y Memoria RAM.
- Combinado con leyes de la lingüística como la Ley de Luhn estas técnicas sirven de
base para operaciones más complejas dentro de otras áreas de NLP, Minería de Texto e 
Information Retrieval.
- En ocasiones es importante tener presente que para funciones muy utilizadas y simples,
podría ser más útil implementar nuestra propia función. Es el ejemplo explicado en la
sección de rendimiento y NTLK ngrams.

Volver al [*Índice*](#indice).

<a id='ejercicios'></a>

## Ejercicios

* **Ejercicio 1:** Implemente un algoritmo que dado un texto cuente las oraciones que hay en él.
Pruebe este algoritmo con el texto pre-procesado y sin procesar.
* **Ejercicio 2:** Implemente una tokenización de textos en inglés que transforme las contracciones.
Cuente el número de palabras únicas con esta implementación y compare con la estudiada
en clase.
* **Ejercicio 3:** Re-implemente la tokenización estudiada en clases, pero donde el algoritmo sea capaz
de entender que *The* y *the* son la misma palabra.
* **Ejercicio 4:** Proponga un nuevo conjunto de stopwords, diseñe una ecuación para calcularlos
automáticamente. (ver la [Ley de Luhn](03.1b-Ley-de-Luhn.ipynb))
* **Ejercicio 5:** Encuentre en internet palabras con raíces linguísticas o lemas poco comunes. Pruebe
los resultados con el steeming de NLTK. Rediseñe el algoritmo de Poter para que reduzca
bien las palabras encontradas por usted.
* **Ejercicio 6:** Proponga una lematización utilizando diccionarios de palabras existentes en internet.
* **Ejercicio 7:** Lea el capítulo de máquinas de estado de Jurafsky, e implemente una máquina parecida
en python.
* **Ejercicio 8:** Utilice los diccionarios de libreoffice de afijos, infijos, etc e implemente un 
lematizador que utilice este recurso.

Volver al [*Índice*](#indice).

<a id='referencias'></a>
## Referencias

<a id='Bird2009'></a>
[1] _[Bird2009]_ Steven Bird, Ewan Klain & Edward Loper,. 
Book **Natural Language Processing with Python**. 2009. 
p. 10 **ISBN**: 978-0-596-51649-9

<a id='Luhn1958'></a>
[2] _[Luhn1958]_ H.P. Luhn. Paper **The Automatic Creation of Literature Abstract**. 
*IBM Journal*, 1958.

<a id='Gorelick2014'></a>
[3] _[Gorelick2014]_ Micha Gorelick & Ian Ozsvald. Book **High Performance Python**. 
O'Reilly. 2014. **ISBN**: 978-1-449-36159-4

<a id='Avello2016'></a>
[3] _[Avello2016]_ Alexander Avello. Thesis **Módulo QtNLP_Wordnet para la aplicación QtNLP**. 
UCLV. 2016.

<a id='alphabetic_index'></a>
## Índice Alfabético

<a id='token'></a>
**Token**: señal, indicio, muestra. Se usa generalmente para referirse a la unidad
más pequeña de procesamiento: palabras, fonemas, n-grams, etc..


Volver al [*Índice*](#indice).